In [1]:
import librosa  #audio n music retrieval systems
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
## chroma: Pertains to the 12 different pitch classes
## mel: Mel Spectrogram Frequency

In [28]:
#extracting mfcc, chroma, mel

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [29]:
#Emotions in the Speech_Train dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [30]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/home/dhruv/Speech_Train/Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [31]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [32]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [41]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [42]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [43]:
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [44]:
y_pred=model.predict(x_test)

In [45]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.19%


In [46]:
y_pred

array(['happy', 'fearful', 'disgust', 'calm', 'fearful', 'fearful',
       'calm', 'fearful', 'disgust', 'fearful', 'happy', 'disgust',
       'calm', 'fearful', 'fearful', 'disgust', 'disgust', 'calm',
       'disgust', 'disgust', 'fearful', 'fearful', 'calm', 'disgust',
       'fearful', 'calm', 'disgust', 'happy', 'calm', 'disgust',
       'disgust', 'disgust', 'disgust', 'happy', 'calm', 'disgust',
       'happy', 'happy', 'disgust', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'happy', 'disgust', 'fearful', 'fearful', 'calm',
       'disgust', 'fearful', 'fearful', 'fearful', 'happy', 'fearful',
       'happy', 'disgust', 'fearful', 'disgust', 'disgust', 'calm',
       'calm', 'disgust', 'disgust', 'fearful', 'fearful', 'fearful',
       'disgust', 'happy', 'disgust', 'disgust', 'fearful', 'calm',
       'disgust', 'calm', 'fearful', 'fearful', 'disgust', 'disgust',
       'fearful', 'disgust', 'disgust', 'disgust', 'happy', 'disgust',
       'disgust', 'fearful', 'disgust

In [52]:
x_test.shape

(192, 180)

In [57]:
f = extract_feature("/home/dhruv/output.wav",mfcc=True,chroma=True,mel=True)

In [58]:
model.predict([f])

array(['happy'], dtype='<U7')